In [ ]:
import torch
import dataset
from torch.utils.data import DataLoader
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from unet_model import UNet

In [32]:
def experiment(test='herschel'):
    test = dataset.Permafrost(['herschel'])

In [33]:
learning_rate = 0.1

dataloader = DataLoader(test, 4)
model = UNet(n_channels =5, n_classes=2)
optimizer = SGD(model.parameters(), learning_rate)

In [ ]:
for (data, labels) in dataloader:
    data, labels = data.cuda(), labels.cuda()
    pred = model(data)
    loss = CrossEntropyLoss(pred, labels)
    loss.backward()
    optimizer.step()